In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
tips = sns.load_dataset('tips')
tips.head(3)

total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3

In [3]:
tips_bill = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()
tips_tip = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()

del tips_bill['tip']
del tips_tip['total_bill']

In [4]:
tips_bill

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [5]:
tips_bill

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [6]:
pd.merge?

In [7]:
# we can merge on the indexes
pd.merge(tips_bill, tips_tip, 
         right_index=True, left_index=True)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [8]:
#we can reset indexes and then merge on the columns - perhaps the easiest way
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index(),
    on=['sex', 'smoker']
)

sex smoker  total_bill     tip
0    Male    Yes     1337.07  183.07
1    Male     No     1919.75  302.00
2  Female    Yes      593.27   96.74
3  Female     No      977.68  149.77

In [9]:
# it can actually infer the above - but be very careful with this
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index()
)

sex smoker  total_bill     tip
0    Male    Yes     1337.07  183.07
1    Male     No     1919.75  302.00
2  Female    Yes      593.27   96.74
3  Female     No      977.68  149.77

In [10]:
# it can merge on partial column and index
pd.merge(
    tips_bill.reset_index(), 
    tips_tip,
    left_on=['sex', 'smoker'],
    right_index=True
)

sex smoker  total_bill     tip
0    Male    Yes     1337.07  183.07
1    Male     No     1919.75  302.00
2  Female    Yes      593.27   96.74
3  Female     No      977.68  149.77

In [11]:
#it can do interesting combinations
tips_bill_strange = tips_bill.reset_index(level=0)
tips_bill_strange

sex  total_bill
smoker                    
Yes       Male     1337.07
No        Male     1919.75
Yes     Female      593.27
No      Female      977.68

In [12]:
pd.merge(
    tips_tip.reset_index(), 
    tips_bill_strange,
    on=['sex', 'smoker']
)

sex smoker     tip  total_bill
0    Male    Yes  183.07     1337.07
1    Male     No  302.00     1919.75
2  Female    Yes   96.74      593.27
3  Female     No  149.77      977.68

In [13]:
pd.merge(
    tips_tip.reset_index(), 
    tips_bill_strange,
    on=['sex', 'smoker']
)

sex smoker     tip  total_bill
0    Male    Yes  183.07     1337.07
1    Male     No  302.00     1919.75
2  Female    Yes   96.74      593.27
3  Female     No  149.77      977.68

In [14]:
# we can do any SQL-like functionality
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2),
    how='left'
)

sex smoker  total_bill     tip
0    Male    Yes     1337.07  183.07
1    Male     No     1919.75  302.00
2  Female    Yes      593.27     NaN
3  Female     No      977.68     NaN

In [15]:
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2),
    how='inner'
)

sex smoker  total_bill     tip
0  Male    Yes     1337.07  183.07
1  Male     No     1919.75  302.00

In [16]:
# and if you add an indicator...
pd.merge(
    tips_bill.reset_index().tail(3), 
    tips_tip.reset_index().head(3),
    how='outer',
    indicator=True
)

sex smoker  total_bill     tip      _merge
0    Male     No     1919.75  302.00        both
1  Female    Yes      593.27   96.74        both
2  Female     No      977.68     NaN   left_only
3    Male    Yes         NaN  183.07  right_only

In [17]:
# it can handle columns with the same name
pd.merge(tips_bill, 
         tips_bill, 
         right_index=True, 
         left_index=True,
         suffixes=('_left', '_right')
)

total_bill_left  total_bill_right
sex    smoker                                   
Male   Yes             1337.07           1337.07
       No              1919.75           1919.75
Female Yes              593.27            593.27
       No               977.68            977.68

In [18]:
# this adds the dataframes together row wise
pd.concat([tips_bill, tips_bill, tips_tip], sort=False)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes            NaN  183.07
       No             NaN  302.00
Female Yes            NaN   96.74
       No             NaN  149.77

In [19]:
# this does it column wise
pd.concat([tips_bill, tips_tip], axis=1)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77